### **Definir el directorio de trabajo**
Define el directorio de trabajo como la carpeta `data` de la carpeta compartida `DS4A-Team12` de Drive.
Este directorio debe contener el archivo de datos original `Sociodemografico.zip` (5945450 registros, 140 MB) en el directorio `raw_data` y el diccionario de datos `dtypes_sociodemo.json` en el directorio `data_dictionaries`.


In [ ]:
import os
import sys
from google.colab import drive 
# Enlazar a la carpeta 'data'
drive.mount('/content/drive')
os.chdir('/content/drive/MyDrive/Colab Notebooks/ICBF/data')
sys.path.insert(0, '../scripts/0_utils')
!pwd

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.activity.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fexperimentsandconfigs%20https%3a%2f%2fwww.googleapis.com%2fauth%2fphotos.native&response_type=code

Enter your authorization code:
4/1AX4XfWh7lV9K3A1GqpTO8-Dd9M5uY3U2McY8_LZiS3qGGnEamYjvqjhO64o
Mounted at /content/drive
/content/drive/MyDrive/Colab Notebooks/ICBF/data


In [ ]:
# Librerias relevantes
import time
import pandas as pd
import json
import re
from utils import replace_log

### **Lectura del archivo**

In [ ]:
t0 = time.time()
with open('datatypes_dictionaries/dtypes_sociodemo.json', 'r') as file:
  dtypes_soc = json.load(file)
# Lee el archivo
soc = pd.read_csv('raw_data/Sociodemografico.zip', sep ='|', header=0,
                  names=dtypes_soc.keys(), encoding='cp1252')
print(f'Archivo leído: {time.time() - t0:.2f} s.')
soc.info()

Archivo leído: 44.04 s.
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5945350 entries, 0 to 5945349
Data columns (total 68 columns):
 #   Column                     Dtype  
---  ------                     -----  
 0   Cod_clase                  int64  
 1   Ind_grupo_sisben_4         object 
 2   Ind_nivel_sisben_4         int64  
 3   Tip_vivienda               int64  
 4   Ind_tiene_energia          int64  
 5   Ind_tiene_alcantarillado   int64  
 6   Ind_tiene_gas              int64  
 7   Ind_tiene_recoleccion      int64  
 8   Ind_tiene_acueducto        int64  
 9   n_hogares_vivienda         int64  
 10  Tip_ocupa_vivienda         int64  
 11  Ind_tiene_cocina           int64  
 12  Ind_tiene_nevera           int64  
 13  Ind_tiene_lavadora         int64  
 14  Ind_tiene_pc               int64  
 15  Ind_tiene_internet         int64  
 16  n_personas_hogar           int64  
 17  Sexo                       float64
 18  Ind_ninguna_discapac       float64
 19  Tip_seg_social    

### **Preprocesamiento**
- Reemplazar algunas fechas de nacimiento futuras (`'FechaNacimiento'` > año 2900, seguramente errores de digitación) por sus equivalentes más probables, para permitir la asignación del tipo `datetime64[ns]`.

In [ ]:
# Eliminar fechas absurdas (>2900)
log = list()
soc['FechaNacimiento'] = replace_log(soc, col='FechaNacimiento', col_id='Id', 
                                     map={'^2[89]': '20'}, log=log_fechas)

In [ ]:
# Asigna a cada variable según su tipo (modo ahorro de memoria)
# Contiene tipos de datos incompatibles con parquet
soc['Sexo'] = soc['Sexo'].astype('Int8')
soc = soc.astype(dtypes_soc)
# Establecer el orden de las categorías de las variables ordinales según su orden natural   
for col in cat_ord:
  soc[col] = soc[col].apply(lambda x: x.cat.as_ordered())
soc.info()
print(f'Preprocesado: {time.time() - t0:.2f} s.')

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5945350 entries, 0 to 5945349
Data columns (total 68 columns):
 #   Column                     Dtype         
---  ------                     -----         
 0   Cod_clase                  category      
 1   Ind_grupo_sisben_4         category      
 2   Ind_nivel_sisben_4         category      
 3   Tip_vivienda               category      
 4   Ind_tiene_energia          category      
 5   Ind_tiene_alcantarillado   category      
 6   Ind_tiene_gas              category      
 7   Ind_tiene_recoleccion      category      
 8   Ind_tiene_acueducto        category      
 9   n_hogares_vivienda         UInt8         
 10  Tip_ocupa_vivienda         category      
 11  Ind_tiene_cocina           category      
 12  Ind_tiene_nevera           category      
 13  Ind_tiene_lavadora         category      
 14  Ind_tiene_pc               category      
 15  Ind_tiene_internet         category      
 16  n_personas_hogar           UInt8    

### **Guardado**
Guarda el dataframes preprocesado en el archivo en formato binario comprimido `sociodemo.parquet` (143 MB), que permite guardar la estructura de los tipos de datos, en el directorio `preprocessed_data`.

In [ ]:
# Escribe un archivo con los cambios generados
with open('log_files/log_sociodemo_1.json', 'w') as log_file:
  log_file.write('Preprocesamiento_Sociodemográfico:\n')
  for line in log_fechas:
    log_file.write('\t' + line + '\n')
 
# Guardar el dataframe preprocesado en un archivo parquet
soc.to_parquet('preprocessed_data/sociodemo.parquet')
print(f'Nuevo archivo guardado: {time.time() - t0:.2f} s.')

Nuevo archivo guardado: 71.31 s.
